# Unit test data
## Create small image for function test
```python
ff = os.path.join(im_dir, full_im)
os_obj = openslide.OpenSlide(ff)
print(os_obj.dimensions)
small_im = os_obj.associated_images['thumbnail']
print(small_im.size)
small_im_name = 'CMU-1-small_im.png'
small_im_full_name = os.path.join(im_dir, small_im_name)
small_im.save(small_im_full_name)
```
(2220, 2967) <br>
(574, 768) <br>
****
## Test: *get_sample_selection_mask(small_im, patch_select_method)*


In [1]:
def get_sample_selection_mask(small_im, patch_select_method):
    """ get an image mask """
    mask_im = None
    
    if patch_select_method == 'threshold_rgb2lab':
        thresh = 80
        np_img = np.array(small_im.convert('RGB'))
        np_img = rgb2lab(np_img)
        np_img = np_img[:,:,0]
        mask_im = np.array(np_img) < thresh
        mask_im = PIL.Image.fromarray(np.uint8(mask_im) * 255)
        
    elif patch_select_method == 'threshold_otsu':
        grey_thumbnail = np.array(small_im.convert('L'))
        thresh = threshold_otsu(grey_thumbnail)
        mask_im = np.array(grey_thumbnail) < thresh
        mask_im = PIL.Image.fromarray(np.uint8(mask_im) * 255)

    else:
        print('patch_select_method %s not implemented'%(patch_select_method))

    return mask_im


"""
                define & import, set-check the input OpenSlide file
"""

import os

import numpy as np

from skimage.filters import threshold_otsu
from skimage.color import rgb2lab
import openslide

import PIL

im_dir = '../data/images/'
full_im = 'CMU-1-Small-Region.svs'
full_im_name = os.path.join(im_dir, full_im)
os.listdir(im_dir)

['threshold_rgb2lab_mask.png',
 'threshold_otsu_mask.tif',
 '.DS_Store',
 'threshold_otsu_mask.png',
 'threshold_rgb2lab_mask.tif',
 'CMU-1-small_im.png',
 'CMU-1-small_im.tif',
 'CMU-1-Small-Region.svs']

In [ ]:
"""
                    open the OpenSlide file, get the associated image called 'thumbnail'
"""
os_obj = openslide.OpenSlide(full_im_name)
small_im = os_obj.associated_images['thumbnail']
print((type(small_im), small_im.size))
os_obj.close()

os.listdir(im_dir)

## Make and write the input thumbnail & mask result images

In [ ]:
"""
                    process and write the result images,
                    delete the image to set test condition
                    
"""
#                                           Write the thumbnail input image
small_im_name = 'CMU-1-small_im.png'
small_im_full_name = os.path.join(im_dir, small_im_name)
small_im.save(small_im_full_name)

patch_select_method_list = ['threshold_rgb2lab', 'threshold_otsu']

#                                           Write the otsu mask
otsu_mask_name = os.path.join(im_dir, 'threshold_otsu_mask.tif')
mask_otsu = get_sample_selection_mask(small_im, patch_select_method_list[1])
mask_otsu.save(otsu_mask_name)

#                                           Write the rgb mask
rgb_mask_name = os.path.join(im_dir, 'threshold_rgb2lab_mask.tif')
mask_rgb = get_sample_selection_mask(small_im, patch_select_method_list[0])
mask_rgb.save(rgb_mask_name)

del small_im

os.listdir(im_dir)

## test: *get_sample_selection_mask(small_im, patch_select_method)*
### Setup the test

In [3]:
patch_select_method_list = ['threshold_rgb2lab', 'threshold_otsu']

small_im_name = 'CMU-1-small_im.tif'
small_im_full_name = os.path.join(im_dir, small_im_name)
small_im = PIL.Image.open(small_im_full_name)

otsu_mask_name = os.path.join(im_dir, 'threshold_otsu_mask.tif')
mask_otsu = get_sample_selection_mask(small_im, patch_select_method_list[1])
otsu_mask_truth = PIL.Image.open(otsu_mask_name)

rgb_mask_name = os.path.join(im_dir, 'threshold_rgb2lab_mask.tif')
mask_rgb = get_sample_selection_mask(small_im, patch_select_method_list[0])
rgb_mask_truth = PIL.Image.open(rgb_mask_name)

## Call each method with the opened input

In [8]:
mask_otsu = get_sample_selection_mask(small_im, patch_select_method_list[1])

# convert to arrays for comparison
otsu_truth_arr = np.array(otsu_mask_truth)
otsu_funct_arr = np.array(mask_otsu)

# compare sizes
print('size otsu_truth_arr:', otsu_truth_arr.shape, '\nsize otsu_funct_arr:', otsu_funct_arr.shape)
# show the sum of the numerical difference 
print('difference =', (otsu_truth_arr - otsu_funct_arr).sum(), '\n\n')


mask_rgb = get_sample_selection_mask(small_im, patch_select_method_list[0])

# convert to arrays for comparison
rgb_truth_arr = np.array(rgb_mask_truth)
rgb_funct_arr = np.array(mask_rgb)

# compare sizes
print('size rgb_truth_arr:', rgb_truth_arr.shape, '\nsize rgb_funct_arr:', rgb_funct_arr.shape)
# show the sum of the numerical difference 
print('difference =', (rgb_truth_arr - rgb_funct_arr).sum())

size otsu_truth_arr: (768, 574) 
size otsu_funct_arr: (768, 574)
difference = 0 


size rgb_truth_arr: (768, 574) 
size rgb_funct_arr: (768, 574)
difference = 0


In [ ]:
#                                   delete the images as in unittest:
del mask_otsu
del mask_rgb
del small_im
del rgb_mask_truth
del otsu_mask_truth

#                                   delete the arrays
del otsu_truth_arr
del otsu_funct_arr
del rgb_truth_arr
del rgb_funct_arr

In [ ]:
"""
                                    Check namespace remainder
"""
%whos